In [67]:
import psycopg2

# connect to database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="example")


In [68]:
# create a cursor
cur = conn.cursor()

# execute a statement
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()

print('PostgreSQL database version:')
print(db_version)

# close the communication with the PostgreSQL
cur.close()

PostgreSQL database version:
('PostgreSQL 15.4 (Debian 15.4-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [50]:
def create_tables():
    cur=conn.cursor()

    create_table_commands=(
        """
            CREATE TABLE filings (
                filing_id varchar(255) PRIMARY KEY,
                cik int,
                filer_name varchar(255),
                period_of_report date
            )
        """,
        """
            CREATE TABLE holdings (
                filing_id varchar(255),
                name_of_issuer varchar(255),
                cusip varchar(255),
                title_of_class varchar(255),
                value bigint,
                shares int,
                put_call varchar(255)
            )
        """, 
        """
            CREATE TABLE holding_infos (
                cusip varchar(255),
                security_name varchar(255),
                ticker varchar(50),
                exchange_code varchar(10),
                security_type varchar(50)
            )
        """
    )

    for command in create_table_commands:
        cur.execute(command)
    cur.close()
    conn.commit()

In [51]:
create_tables()

DuplicateTable: relation "filings" already exists


In [52]:
pip install sec-api

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 11.1/11.1 MB 10.9 MB/s eta 0:00:00
  Using cached numpy-1.25.2-cp310-cp310-win_amd64.whl (15.6 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)


In [62]:
from sec_api import QueryApi
import pandas as pd

queryApi=QueryApi(api_key="YOUR API KEY")

def get_13f_filings(start=0):
    print(f"Getting next 13F batch starting at {start}")

    query={
        "query":{
            "query_string":{
                "query":"formType:\"13F-HR\" AND NOT formtype:\"13F=HR/A\" AND periodOfReport:\"2021-06-30\""
                
            }
        },
        "from":start,
        "size": "10",
        "sort":[{"filedAt":{"order":"desc"}}]
    }
    response=queryApi.get_filings(query)
    return response['filings']

filings_batch=get_13f_filings(10)

holdings_example=pd.json_normalize(filings_batch[0]['holdings'])






Getting next 13F batch starting at 10


In [54]:
print(holdings_example)

          cusip ticker      cik otherManager investmentDiscretion  \
0     000360206   AAON   824142            1                 DFND   
1     000361105    AIR     1750            1                 DFND   
2     00081T108   ACCO   712034            1                 DFND   
3     000957100    ABM   771497            1                 DFND   
4     001055102    AFL     4977            1                 DFND   
...         ...    ...      ...          ...                  ...   
2255  G1151C101    ACN  1467373            1                 DFND   
2256  G3223R108     RE  1095073            1                 DFND   
2257  H1467J104     CB   896159            1                 DFND   
2258  H84989104    TEL  1385157            1                 DFND   
2259  V7780T103    RCL   884887            1                 DFND   

               nameOfIssuer     value    titleOfClass  votingAuthority.Sole  \
0                  AAON INC    895600  COM PAR $0.004                     0   
1            

In [55]:
display(holdings_example)

,cusip,ticker,cik,otherManager,investmentDiscretion,nameOfIssuer,value,titleOfClass,votingAuthority.Sole,votingAuthority.Shared,votingAuthority.None,shrsOrPrnAmt.sshPrnamtType,shrsOrPrnAmt.sshPrnamt
0,000360206,AAON,824142,1,DFND,AAON INC,895600,COM PAR $0.004,0,14309,0,SH,14309
1,000361105,AIR,1750,1,DFND,AAR CORP,537153,COM,0,13862,0,SH,13862
2,00081T108,ACCO,712034,1,DFND,ACCO BRANDS CORP,186701,COM,0,21634,0,SH,21634
3,000957100,ABM,771497,1,DFND,ABM INDS INC,1103916,COM,0,24891,0,SH,24891
4,001055102,AFL,4977,1,DFND,AFLAC INC,7753548,COM,0,144494,0,SH,144494
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,G1151C101,ACN,1467373,1,DFND,ACCENTURE PLC IRELAND,43443792,SHS CLASS A,0,147372,0,SH,147372
2256,G3223R108,RE,1095073,1,DFND,EVEREST RE GROUP LTD,2341929,COM,0,9293,0,SH,9293
2257,H1467J104,CB,896159,1,DFND,CHUBB LIMITED,16570767,COM,0,104258,0,SH,104258
2258,H84989104,TEL,1385157,1,DFND,TE CONNECTIVITY LTD,10351678,REG SHS,0,76560,0,SH,76560


In [56]:
type(filings_batch)


list

In [57]:
type(filings_batch[0])

dict

In [63]:
def save_to_db(filings):
    cur = conn.cursor()

    for filing in filings:
        if len(filing['holdings']) == 0:
            continue
        
        insert_commands = (
            """
                INSERT INTO filings (
                    filing_id, 
                    cik,
                    filer_name,
                    period_of_report
                ) 
                VALUES (%s, %s, %s, %s)
            """,
            """
                INSERT INTO holdings (
                    filing_id, 
                    name_of_issuer,
                    cusip,
                    title_of_class,
                    value,
                    shares,
                    put_call
                ) 
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
        )
        
        filing_values = (
            filing['id'],
            filing['cik'], 
            filing['companyName'].upper(), # convert all names to upper case to make unique search easier
            filing['periodOfReport']
        )
        
        cur.execute(insert_commands[0], filing_values)
        
        for holding in filing['holdings']:       
            holding_values = (
                filing['id'], 
                holding['nameOfIssuer'].upper(), # convert all names to upper case to make unique search easier
                holding['cusip'],
                holding['titleOfClass'],
                holding['value'],
                holding['shrsOrPrnAmt']['sshPrnamt'],
                holding['putCall'] if 'putCall' in holding else '',
            )
                    
            cur.execute(insert_commands[1], holding_values)
        
    cur.close()
    conn.commit()


        




In [65]:
save_to_db(filings_batch)

In [66]:
cur = conn.cursor()
cur.execute("SELECT count(*) as COUNTER FROM holdings")
counter = cur.fetchone()
print(counter[0])
cur.close()

2628


In [69]:
# cusip-mapper.py
import psycopg2
import requests

# connect to database


def get_all_cusips():
    cur = conn.cursor()

    cur.execute("SELECT DISTINCT cusip FROM holdings WHERE NOT cusip LIKE '000%'")

    rows = cur.fetchall()

    return [row[0] for row in rows]


def cusip_to_query(cusip):
    return {"idType": "ID_CUSIP", "idValue": str(cusip)}


def format_response(response):
    if "data" in response and len(response["data"]) != 0:
        match = response["data"][0]
        return match["ticker"], match["name"], match["securityType"]

    return "", "", ""


def cusips_to_tickers(cusips):
    api_endpoint = "https://api.openfigi.com/v3/mapping"
    headers = {"X-OPENFIGI-APIKEY": "YOUR OPENFIGI-APIKEY"}
    query = [cusip_to_query(cusip) for cusip in cusips]

    response = requests.post(api_endpoint, json=query, headers=headers)

    matches = response.json()

    tmp = [format_response(match) for match in matches]

    return [(tpl[0],) + tpl[1] for tpl in list(zip(cusips, tmp))]


def insert_into_db(rows):
    cur = conn.cursor()

    insert_command = """
        INSERT INTO holding_infos (
            cusip,
            ticker,
            security_name,
            security_type
        )
        VALUES (%s, %s, %s, %s)
         """

    try:
        cur.executemany(insert_command, rows)
        cur.close()
        conn.commit()
    finally:
        conn.commit()


def run():
    cusips = get_all_cusips()

    start = 0
    stop = 100

    while start < len(cusips):
        print(start, stop)
        if stop <= len(cusips):
            cusip_batch = cusips[start:stop]
        else:
            cusip_batch = cusips[start:]

        cusips_map = cusips_to_tickers(cusip_batch)

        insert_into_db(cusips_map)

        start = start + 100
        stop = stop + 100

        if start >= len(cusips):
            break

    print("Done")


run()

0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000
1000 1100
1100 1200
1200 1300
1300 1400
1400 1500
1500 1600
1600 1700
1700 1800
1800 1900
1900 2000
2000 2100
2100 2200
2200 2300
2300 2400
Done


In [81]:
cur = conn.cursor()

cur.execute("""
SELECT cusip,security_name,ticker, FROM "holding_infos"
LIMIT 4
""")

result = cur.fetchall()    

cur.close()
print(result)

[('319829107', 'FIRST COMMONWEALTH FINL CORP', 'FCF', None, 'Common Stock'), ('044186104', 'ASHLAND INC', 'ASH', None, 'Common Stock'), ('24869P104', "DENNY'S CORP", 'DENN', None, 'Common Stock'), ('609839105', 'MONOLITHIC POWER SYSTEMS INC', 'MPWR', None, 'Common Stock')]


In [82]:
display(pd.DataFrame(result))

,0,1,2,3,4
0,319829107,FIRST COMMONWEALTH FINL CORP,FCF,None,Common Stock
1,044186104,ASHLAND INC,ASH,None,Common Stock
2,24869P104,DENNY'S CORP,DENN,None,Common Stock
3,609839105,MONOLITHIC POWER SYSTEMS INC,MPWR,None,Common Stock


In [80]:
conn.commit()
cur = conn.cursor()

cur.execute("""
SELECT shares, holdings.cusip, security_name, ticker, period_of_report, holdings.filing_id
FROM "holdings"
INNER JOIN filings
ON filings.filing_id = holdings.filing_id
INNER JOIN holding_infos
ON holdings.cusip = holding_infos.cusip
WHERE (period_of_report = '2021-06-30' or period_of_report = '2021-03-31') and cik = '1529735'
""")

rows = cur.fetchall()    

cur.close()

buffet_holdings = pd.DataFrame(rows, columns =['Shares', 
                                               'CUSIP', 
                                               'SecurityName', 
                                               'Ticker', 
                                               'PeriodOfReport', 
                                               'FilingId'])
display(buffet_holdings)

,Shares,CUSIP,SecurityName,Ticker,PeriodOfReport,FilingId
0,34651,319829107,FIRST COMMONWEALTH FINL CORP,FCF,2021-06-30,991a614906196933c27fe0023256d245
1,27301,044186104,ASHLAND INC,ASH,2021-06-30,991a614906196933c27fe0023256d245
2,22809,24869P104,DENNY'S CORP,DENN,2021-06-30,991a614906196933c27fe0023256d245
3,9972,609839105,MONOLITHIC POWER SYSTEMS INC,MPWR,2021-06-30,991a614906196933c27fe0023256d245
4,421248,254687106,WALT DISNEY CO/THE,DIS,2021-06-30,991a614906196933c27fe0023256d245
...,...,...,...,...,...,...
2156,16756,695127100,PACIRA BIOSCIENCES INC,PCRX,2021-06-30,991a614906196933c27fe0023256d245
2157,6422,02913V103,AMERICAN PUBLIC EDUCATION,APEI,2021-06-30,991a614906196933c27fe0023256d245
2158,37060,042735100,ARROW ELECTRONICS INC,ARW,2021-06-30,991a614906196933c27fe0023256d245
2159,11584,535219109,LINDBLAD EXPEDITIONS HOLDING,LIND,2021-06-30,991a614906196933c27fe0023256d245
